In [0]:
#NO NEED TO RUN
#Code to split "data" folder into "train", "val", and "test" folders from "Data" folder on Google Drive

#pip install split_folders
#import split_folders

#split_folders.ratio('/content/drive/My Drive/Data', output='/content/drive/My Drive/test_train_validate', seed=133, ratio=(.8, .15, .05))

<font size="5"> 
IMPORTANT: ***Click Kernel>Change kernel>solar-kernel to select kernel created in readme***

<font size="5"> 
**The cells below download the validation and test images from the pre-split dataset, from which Keras data generators are created.**
<br/><br/>
<font size="4"> 
(Code used to split original dataset is shown above.)

<font size="4"> 
    
**First, the validation and test image sets must be downloaded.**
        <br/><br/>
Link for test set: https://www.dropbox.com/sh/3yilkr8dpbbbfui/AABrLOrvMXxjvmDRdIXDMN2ha?dl=0
        <br/><br/>
Link for validation set: https://www.dropbox.com/sh/lgf0j8s3hbdi5i0/AAC3jEhaoBRm8zboUDRF6lFla?dl=0

<font size="4">
    
**Folders should be placed in the same directory as this notebook.**

In [0]:
#Loading in images from train_test_validate

from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Create data generator
datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True)

In [0]:
#Validation generator
validation_generator = datagen.flow_from_directory(
        '/content/drive/My Drive/test_train_validate/val',
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary',
        shuffle=False)

Found 2470 images belonging to 2 classes.


In [0]:
#Test generator
test_generator = datagen.flow_from_directory(
        '/content/drive/My Drive/test_train_validate/test',
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary',
        shuffle=False)

Found 826 images belonging to 2 classes.


<font size="5"> 
    
NOT RECOMMENDED: Run the following cells to load in baseline Inception V3 model, re-train single layer across 2 epochs, and then unfreeze all layers beyond "mixed6" and re-train across 42 epochs
<br/><br/>
    
***Training will take a signifigant amount of time due to the size of the training set.*** 
<br/><br/>
<font size="4"> 
    
**It is recommended to instead skip to the markdown cell below (towards the end of the notebook) where the pre-trained model weights are loaded in, in order to avoid having to re-run the training process.**

<font size="4"> 
Download training set: https://www.dropbox.com/sh/gcv9ljkf4la930s/AADg8jfItbZzuS3XM81hIAQXa?dl=0

In [ ]:
#Train generator
train_generator = datagen.flow_from_directory(
        '/content/drive/My Drive/test_train_validate/train',
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary')

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import metrics

#Loading in Inception V3 pre-trained model
pre_trained_model=InceptionV3(input_shape=(300,300,3),
                              include_top=False,
                              weights='imagenet')
#Freezing layers
for layer in pre_trained_model.layers:
    layer.trainable=False
#Adding mixed7 layer
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output
x = layers.Flatten()(last_output)
#Adding fully connected layer
x = layers.Dense(1024, activation='relu')(x)
# Set dropout rate
x = layers.Dropout(0.2)(x) #Default is 0.2
x = layers.Dense(1, activation='sigmoid')(x)
#Setting metrics
metrics = [
  metrics.BinaryAccuracy(name='accuracy'),
  metrics.Precision(name='precision'),
  metrics.Recall(name='recall'),
  metrics.TruePositives(name='tp'),
  metrics.FalsePositives(name='fp'),
  metrics.TrueNegatives(name='tn'),
  metrics.FalseNegatives(name='fn')]
#Compiling model: here we use an RMSprop optimizer
model= Model(pre_trained_model.input,x)
model.compile(optimizer=RMSprop(lr=0.001,momentum=0.9,decay=0.9),
              loss='binary_crossentropy',
              metrics=metrics)

In [0]:
#Fit model with 2 epochs, allowing single layer of Inception V3 model to be re-trained
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

Epoch 1/2
659/659 [==============================] - 7093s 11s/step - loss: 33.4850 - accuracy: 0.8193 - precision: 0.4581 - recall: 0.7430 - tp: 1558.0000 - fp: 1843.0000 - tn: 9240.0000 - fn: 539.0000 - val_loss: 15.1891 - val_accuracy: 0.8354 - val_precision: 0.4812 - val_recall: 0.7363 - val_tp: 282.0000 - val_fp: 304.0000 - val_tn: 1773.0000 - val_fn: 101.0000
Epoch 2/2
659/659 [==============================] - 156s 237ms/step - loss: 29.3147 - accuracy: 0.8304 - precision: 0.4798 - recall: 0.7632 - tp: 1602.0000 - fp: 1737.0000 - tn: 9339.0000 - fn: 497.0000 - val_loss: 12.9609 - val_accuracy: 0.8463 - val_precision: 0.5045 - val_recall: 0.7389 - val_tp: 283.0000 - val_fp: 278.0000 - val_tn: 1799.0000 - val_fn: 100.0000


In [0]:
#Fine tune convolutional layers
unfreeze = False
#Unfreeze all models after "mixed6"
for layer in pre_trained_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed6':
        unfreeze = True
#Compile model again, this time using a Stochastic Gradient Descent (SGD) optimizer with a low learning rate (0.000075)
model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=0.000075),
              metrics=metrics)

The code below trains the model across 42 epochs. Training was done over multiple cells, and the model was saved at several points thorughout the training process.

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

Epoch 1/2
659/659 [==============================] - 158s 239ms/step - loss: 19.0429 - accuracy: 0.8517 - precision: 0.5207 - recall: 0.8070 - tp: 1999.0000 - fp: 1840.0000 - tn: 11318.0000 - fn: 478.0000 - val_loss: 3.0517 - val_accuracy: 0.9455 - val_precision: 0.8430 - val_recall: 0.7990 - val_tp: 306.0000 - val_fp: 57.0000 - val_tn: 2020.0000 - val_fn: 77.0000
Epoch 2/2
659/659 [==============================] - 154s 234ms/step - loss: 8.4704 - accuracy: 0.8935 - precision: 0.6181 - recall: 0.8701 - tp: 1829.0000 - fp: 1130.0000 - tn: 9943.0000 - fn: 273.0000 - val_loss: 2.7127 - val_accuracy: 0.9549 - val_precision: 0.9359 - val_recall: 0.7624 - val_tp: 292.0000 - val_fp: 20.0000 - val_tn: 2057.0000 - val_fn: 91.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 155s 235ms/step - loss: 5.6536 - accuracy: 0.9121 - precision: 0.6671 - recall: 0.8926 - tp: 1870.0000 - fp: 933.0000 - tn: 10147.0000 - fn: 225.0000 - val_loss: 1.7409 - val_accuracy: 0.9634 - val_precision: 0.9322 - val_recall: 0.8251 - val_tp: 316.0000 - val_fp: 23.0000 - val_tn: 2054.0000 - val_fn: 67.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 155s 235ms/step - loss: 4.5751 - accuracy: 0.9170 - precision: 0.6817 - recall: 0.8964 - tp: 1878.0000 - fp: 877.0000 - tn: 10203.0000 - fn: 217.0000 - val_loss: 1.4327 - val_accuracy: 0.9573 - val_precision: 0.8174 - val_recall: 0.9347 - val_tp: 358.0000 - val_fp: 80.0000 - val_tn: 1997.0000 - val_fn: 25.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4}) 

659/659 [==============================] - 156s 237ms/step - loss: 3.7298 - accuracy: 0.9301 - precision: 0.7219 - recall: 0.9133 - tp: 1918.0000 - fp: 739.0000 - tn: 10336.0000 - fn: 182.0000 - val_loss: 6.6553 - val_accuracy: 0.7561 - val_precision: 0.3883 - val_recall: 0.9843 - val_tp: 377.0000 - val_fp: 594.0000 - val_tn: 1483.0000 - val_fn: 6.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 159s 241ms/step - loss: 3.3032 - accuracy: 0.9327 - precision: 0.7293 - recall: 0.9175 - tp: 1924.0000 - fp: 714.0000 - tn: 10364.0000 - fn: 173.0000 - val_loss: 1.1789 - val_accuracy: 0.9626 - val_precision: 0.8299 - val_recall: 0.9556 - val_tp: 366.0000 - val_fp: 75.0000 - val_tn: 2002.0000 - val_fn: 17.0000


In [0]:
#model.save('/content/drive/My Drive/Models/solar_model_5_2_v0.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 191s 290ms/step - loss: 3.5545 - accuracy: 0.9312 - precision: 0.7246 - recall: 0.9151 - tp: 1918.0000 - fp: 729.0000 - tn: 10350.0000 - fn: 178.0000 - val_loss: 1.0552 - val_accuracy: 0.9630 - val_precision: 0.8460 - val_recall: 0.9321 - val_tp: 357.0000 - val_fp: 65.0000 - val_tn: 2012.0000 - val_fn: 26.0000


In [0]:
#model.save('solar_model_5_2_v1.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 169s 256ms/step - loss: 2.6690 - accuracy: 0.9425 - precision: 0.7624 - recall: 0.9275 - tp: 1944.0000 - fp: 606.0000 - tn: 10473.0000 - fn: 152.0000 - val_loss: 1.0028 - val_accuracy: 0.9661 - val_precision: 0.8507 - val_recall: 0.9567 - val_tp: 376.0000 - val_fp: 66.0000 - val_tn: 1991.0000 - val_fn: 17.0000


In [0]:
#model.save('solar_model_5_2_v2.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4}) #HERE

659/659 [==============================] - 183s 278ms/step - loss: 2.6579 - accuracy: 0.9415 - precision: 0.7584 - recall: 0.9274 - tp: 1943.0000 - fp: 619.0000 - tn: 10461.0000 - fn: 152.0000 - val_loss: 0.8488 - val_accuracy: 0.9755 - val_precision: 0.9258 - val_recall: 0.9211 - val_tp: 362.0000 - val_fp: 29.0000 - val_tn: 2028.0000 - val_fn: 31.0000


In [0]:
#model.save('solar_model_5_2_v3.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 182s 277ms/step - loss: 2.1026 - accuracy: 0.9495 - precision: 0.7868 - recall: 0.9365 - tp: 1963.0000 - fp: 532.0000 - tn: 10547.0000 - fn: 133.0000 - val_loss: 2.4033 - val_accuracy: 0.8812 - val_precision: 0.5763 - val_recall: 0.9796 - val_tp: 385.0000 - val_fp: 283.0000 - val_tn: 1774.0000 - val_fn: 8.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 161s 245ms/step - loss: 2.3561 - accuracy: 0.9431 - precision: 0.7634 - recall: 0.9309 - tp: 1952.0000 - fp: 605.0000 - tn: 10473.0000 - fn: 145.0000 - val_loss: 1.0326 - val_accuracy: 0.9714 - val_precision: 0.9575 - val_recall: 0.8601 - val_tp: 338.0000 - val_fp: 15.0000 - val_tn: 2042.0000 - val_fn: 55.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 162s 245ms/step - loss: 2.1865 - accuracy: 0.9468 - precision: 0.7776 - recall: 0.9323 - tp: 1954.0000 - fp: 559.0000 - tn: 10520.0000 - fn: 142.0000 - val_loss: 0.9685 - val_accuracy: 0.9616 - val_precision: 0.8286 - val_recall: 0.9593 - val_tp: 377.0000 - val_fp: 78.0000 - val_tn: 1979.0000 - val_fn: 16.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 162s 245ms/step - loss: 2.0566 - accuracy: 0.9493 - precision: 0.7862 - recall: 0.9356 - tp: 1960.0000 - fp: 533.0000 - tn: 10547.0000 - fn: 135.0000 - val_loss: 4.5123 - val_accuracy: 0.8082 - val_precision: 0.4548 - val_recall: 0.9847 - val_tp: 387.0000 - val_fp: 464.0000 - val_tn: 1593.0000 - val_fn: 6.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 162s 246ms/step - loss: 1.7894 - accuracy: 0.9542 - precision: 0.8027 - recall: 0.9437 - tp: 1977.0000 - fp: 486.0000 - tn: 10594.0000 - fn: 118.0000 - val_loss: 0.6941 - val_accuracy: 0.9780 - val_precision: 0.9104 - val_recall: 0.9567 - val_tp: 376.0000 - val_fp: 37.0000 - val_tn: 2020.0000 - val_fn: 17.0000


In [0]:
#model.save('solar_model_5_2_v4.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 168s 255ms/step - loss: 1.5658 - accuracy: 0.9547 - precision: 0.8046 - recall: 0.9447 - tp: 1981.0000 - fp: 481.0000 - tn: 10597.0000 - fn: 116.0000 - val_loss: 0.7630 - val_accuracy: 0.9731 - val_precision: 0.8884 - val_recall: 0.9517 - val_tp: 374.0000 - val_fp: 47.0000 - val_tn: 2010.0000 - val_fn: 19.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 162s 245ms/step - loss: 1.4672 - accuracy: 0.9568 - precision: 0.8138 - recall: 0.9452 - tp: 1984.0000 - fp: 454.0000 - tn: 10622.0000 - fn: 115.0000 - val_loss: 0.7533 - val_accuracy: 0.9727 - val_precision: 0.8844 - val_recall: 0.9542 - val_tp: 375.0000 - val_fp: 49.0000 - val_tn: 2008.0000 - val_fn: 18.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 160s 243ms/step - loss: 1.2607 - accuracy: 0.9624 - precision: 0.8340 - recall: 0.9533 - tp: 2000.0000 - fp: 398.0000 - tn: 10679.0000 - fn: 98.0000 - val_loss: 0.6442 - val_accuracy: 0.9722 - val_precision: 0.8842 - val_recall: 0.9517 - val_tp: 374.0000 - val_fp: 49.0000 - val_tn: 2008.0000 - val_fn: 19.0000


In [0]:
#model.save('solar_model_5_2_v5.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 170s 257ms/step - loss: 1.4366 - accuracy: 0.9565 - precision: 0.8124 - recall: 0.9446 - tp: 1979.0000 - fp: 457.0000 - tn: 10623.0000 - fn: 116.0000 - val_loss: 0.5537 - val_accuracy: 0.9804 - val_precision: 0.9389 - val_recall: 0.9389 - val_tp: 369.0000 - val_fp: 24.0000 - val_tn: 2033.0000 - val_fn: 24.0000


In [0]:
#model.save('solar_model_5_2_v6.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 174s 264ms/step - loss: 1.4835 - accuracy: 0.9577 - precision: 0.8165 - recall: 0.9470 - tp: 1985.0000 - fp: 446.0000 - tn: 10633.0000 - fn: 111.0000 - val_loss: 0.8757 - val_accuracy: 0.9616 - val_precision: 0.8300 - val_recall: 0.9567 - val_tp: 376.0000 - val_fp: 77.0000 - val_tn: 1980.0000 - val_fn: 17.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 181s 275ms/step - loss: 1.3827 - accuracy: 0.9592 - precision: 0.8220 - recall: 0.9500 - tp: 1995.0000 - fp: 432.0000 - tn: 10643.0000 - fn: 105.0000 - val_loss: 0.5214 - val_accuracy: 0.9812 - val_precision: 0.9437 - val_recall: 0.9389 - val_tp: 369.0000 - val_fp: 22.0000 - val_tn: 2035.0000 - val_fn: 24.0000


In [0]:
#model.save('solar_model_5_2_v7.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 179s 272ms/step - loss: 1.1195 - accuracy: 0.9634 - precision: 0.8386 - recall: 0.9538 - tp: 2001.0000 - fp: 385.0000 - tn: 10692.0000 - fn: 97.0000 - val_loss: 2.4117 - val_accuracy: 0.8580 - val_precision: 0.5308 - val_recall: 0.9873 - val_tp: 388.0000 - val_fp: 343.0000 - val_tn: 1714.0000 - val_fn: 5.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 160s 242ms/step - loss: 1.2838 - accuracy: 0.9587 - precision: 0.8200 - recall: 0.9490 - tp: 1991.0000 - fp: 437.0000 - tn: 10640.0000 - fn: 107.0000 - val_loss: 0.5852 - val_accuracy: 0.9788 - val_precision: 0.9273 - val_recall: 0.9415 - val_tp: 370.0000 - val_fp: 29.0000 - val_tn: 2028.0000 - val_fn: 23.0000


In [0]:
#model.save('solar_model_5_2_v8.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 173s 263ms/step - loss: 1.0790 - accuracy: 0.9654 - precision: 0.8457 - recall: 0.9571 - tp: 2006.0000 - fp: 366.0000 - tn: 10713.0000 - fn: 90.0000 - val_loss: 0.7952 - val_accuracy: 0.9763 - val_precision: 0.9772 - val_recall: 0.8728 - val_tp: 343.0000 - val_fp: 8.0000 - val_tn: 2049.0000 - val_fn: 50.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 159s 242ms/step - loss: 0.9888 - accuracy: 0.9668 - precision: 0.8517 - recall: 0.9585 - tp: 2010.0000 - fp: 350.0000 - tn: 10728.0000 - fn: 87.0000 - val_loss: 0.8340 - val_accuracy: 0.9743 - val_precision: 0.9825 - val_recall: 0.8550 - val_tp: 336.0000 - val_fp: 6.0000 - val_tn: 2051.0000 - val_fn: 57.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 161s 244ms/step - loss: 1.0108 - accuracy: 0.9646 - precision: 0.8431 - recall: 0.9557 - tp: 2004.0000 - fp: 373.0000 - tn: 10705.0000 - fn: 93.0000 - val_loss: 0.5916 - val_accuracy: 0.9796 - val_precision: 0.9549 - val_recall: 0.9160 - val_tp: 360.0000 - val_fp: 17.0000 - val_tn: 2040.0000 - val_fn: 33.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 157s 239ms/step - loss: 1.1678 - accuracy: 0.9639 - precision: 0.8404 - recall: 0.9542 - tp: 2001.0000 - fp: 380.0000 - tn: 10698.0000 - fn: 96.0000 - val_loss: 0.6178 - val_accuracy: 0.9816 - val_precision: 0.9703 - val_recall: 0.9135 - val_tp: 359.0000 - val_fp: 11.0000 - val_tn: 2046.0000 - val_fn: 34.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=13,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

Epoch 1/13
659/659 [==============================] - 158s 240ms/step - loss: 0.9178 - accuracy: 0.9700 - precision: 0.8640 - recall: 0.9633 - tp: 2021.0000 - fp: 318.0000 - tn: 10759.0000 - fn: 77.0000 - val_loss: 0.7705 - val_accuracy: 0.9780 - val_precision: 0.9748 - val_recall: 0.8855 - val_tp: 348.0000 - val_fp: 9.0000 - val_tn: 2048.0000 - val_fn: 45.0000
Epoch 2/13
659/659 [==============================] - 156s 236ms/step - loss: 0.8774 - accuracy: 0.9681 - precision: 0.8568 - recall: 0.9594 - tp: 2011.0000 - fp: 336.0000 - tn: 10748.0000 - fn: 85.0000 - val_loss: 0.4942 - val_accuracy: 0.9808 - val_precision: 0.9413 - val_recall: 0.9389 - val_tp: 369.0000 - val_fp: 23.0000 - val_tn: 2034.0000 - val_fn: 24.0000
Epoch 3/13
659/659 [==============================] - 154s 234ms/step - loss: 0.6739 - accuracy: 0.9760 - precision: 0.8891 - recall: 0.9705 - tp: 2037.0000 - fp: 254.0000 - tn: 10822.0000 - fn: 62.0000 - val_loss: 0.5786 - val_accuracy: 0.9829 - val_precision: 0.9631 - 

In [0]:
#model.save('solar_model_5_2_v9.h5')

In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 173s 263ms/step - loss: 0.5583 - accuracy: 0.9762 - precision: 0.8913 - recall: 0.9696 - tp: 2041.0000 - fp: 249.0000 - tn: 10821.0000 - fn: 64.0000 - val_loss: 0.4472 - val_accuracy: 0.9776 - val_precision: 0.9043 - val_recall: 0.9618 - val_tp: 378.0000 - val_fp: 40.0000 - val_tn: 2017.0000 - val_fn: 15.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 158s 240ms/step - loss: 0.6228 - accuracy: 0.9743 - precision: 0.8809 - recall: 0.9689 - tp: 2026.0000 - fp: 274.0000 - tn: 10810.0000 - fn: 65.0000 - val_loss: 0.5919 - val_accuracy: 0.9706 - val_precision: 0.8673 - val_recall: 0.9644 - val_tp: 379.0000 - val_fp: 58.0000 - val_tn: 1999.0000 - val_fn: 14.0000


In [0]:
#Fit model
history = model.fit(
      train_generator,
      steps_per_epoch=13195//20,
      epochs=1,
      validation_data=validation_generator,
      validation_steps=2470//20,
      verbose=1,
      workers=1,
      use_multiprocessing=False,
      class_weight={0:1,
                    1:4})

659/659 [==============================] - 160s 243ms/step - loss: 0.4354 - accuracy: 0.9806 - precision: 0.9082 - recall: 0.9766 - tp: 2048.0000 - fp: 207.0000 - tn: 10871.0000 - fn: 49.0000 - val_loss: 0.4696 - val_accuracy: 0.9837 - val_precision: 0.9561 - val_recall: 0.9415 - val_tp: 370.0000 - val_fp: 17.0000 - val_tn: 2040.0000 - val_fn: 23.0000


In [0]:
#Saving final model
model.save('final_model.h5')

<font size="4"> 
Final model was trained over 42 epochs, shown above.

<font size="5"> 
    
***Continue here if loading in pre-trained model***
<br/><br/>
The code below downloads an h5 file containing the pre-trained model weights, and then loads in the file as a Keras model.

In [3]:
#Download final version of model, used to generate results reported in presentation
!curl -L -o final_model.h5 https://www.dropbox.com/s/mprqnfq3prmqun4/final_model.h5?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  1034    0  1034    0     0    252      0 --:--:--  0:00:04 --:--:--   524
100  901M  100  901M    0     0  2324k      0  0:06:37  0:06:37 --:--:-- 2162k:16  0:10:14 1425k:07:48 1752k0:00:30  0:07:28 1929k:17  0:01:01  0:07:16 1713kk      0  0:06:52  0:02:36  0:04:16 1838k    0  0:06:56  0:03:53  0:03:03  860k:57  0:04:01  0:02:56 2613k  0:02:45 3146k:04:30  0:02:23 3371k0:04:41  0:02:04 3468k  0:04:48  0:01:57 2150k 3057k  2323k      0  0:06:37  0:05:58  0:00:39 1852k    0     0  2332k      0  0:06:35  0:06:07  0:00:28 2993k 0:06:15  0:00:21 1949k 0  0:06:36  0:06:33  0:00:03 2179k


In [0]:
#Load in downloaded model
from tensorflow.keras.models import load_model
loaded_model=load_model('final_model.h5')

<font size="5"> 
    
***The code below generates metrics (accuracy, precision, recall, and confusion matrices) for the validation and test sets.***
    
<font size="4"> 
    
Because the validation results influence the setting of hyperparameters, test results offer the best possible approximation of the model's expected real-world performance.

In [0]:
#Evaluate model

#Generate predictions from validation set
validation_generator.reset()
predictions=loaded_model.predict(validation_generator,steps=2470//20+1,verbose=1)
predictions_classes=predictions>0.5
predictions_classes=predictions_classes.flatten().astype(int)
actual=validation_generator.classes

124/124 [==============================] - 27s 218ms/step


<font size="4"> 
***Validation results*** (validation set contains 2470 images)

In [0]:
from sklearn import metrics
#Validation accuracy
metrics.accuracy_score(actual,predictions_classes)

0.9817813765182186

In [0]:
#Confusion matrix
metrics.confusion_matrix(actual,predictions_classes)

array([[2059,   18],
       [  27,  366]])

In [0]:
#Precision
metrics.precision_score(actual,predictions_classes)

0.953125

In [0]:
#Recall
metrics.recall_score(actual,predictions_classes)

0.9312977099236641

In [0]:
#Generate predictions from test set
test_generator.reset()
predictions=loaded_model.predict(test_generator,steps=826//20+1,verbose=1)
predictions_classes=predictions>0.5
predictions_classes=predictions_classes.flatten().astype(int)
actual=test_generator.classes

42/42 [==============================] - 9s 208ms/step


<font size="4"> 
***Test results*** (test set contains 826 images)

In [0]:
#Test accuracy
metrics.accuracy_score(actual,predictions_classes)

0.9794188861985472

In [0]:
#Confusion matrix
metrics.confusion_matrix(actual,predictions_classes)

array([[685,   9],
       [  8, 124]])

In [0]:
#Precision
metrics.precision_score(actual,predictions_classes)

0.9323308270676691

In [0]:
#Recall
metrics.recall_score(actual,predictions_classes)

0.9393939393939394

In [0]:
#Generate predictions from training set

<font size="4"> 
***Training results (not reported in presentation)***
<br/><br/>
Please note that due to the large size of the training set (13195 images), this code will take significant time to run.

<font size="4"> 
Download training set (if not already done): https://www.dropbox.com/sh/gcv9ljkf4la930s/AADg8jfItbZzuS3XM81hIAQXa?dl=0

In [0]:
#Read in training data with shuffle=False
train_generator_no_shuffle = datagen.flow_from_directory(
        '/content/drive/My Drive/test_train_validate/train',
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary',
        shuffle=False)

Found 13195 images belonging to 2 classes.


In [0]:
train_generator_no_shuffle.reset()
predictions=loaded_model.predict(train_generator_no_shuffle,steps=13195//20+1,verbose=1)
predictions_classes=predictions>0.5
predictions_classes=predictions_classes.flatten().astype(int)
actual=train_generator_no_shuffle.classes

660/660 [==============================] - 153s 232ms/step


In [0]:
#Training accuracy
metrics.accuracy_score(actual,predictions_classes)

0.993179234558545

In [0]:
#Confusion matrix
metrics.confusion_matrix(actual,predictions_classes)

array([[11023,    72],
       [   18,  2082]])

In [0]:
#Precision
metrics.precision_score(actual,predictions_classes)

0.9665738161559888

In [0]:
#Recall
metrics.recall_score(actual,predictions_classes)

0.9914285714285714

<font size="4">
    
**Create diagram of network**

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(loaded_model, to_file='solar_model_plot.png')

<font size="5"> 
    
**The code below was used to generate solar predictions (both binary classifications and probabilities) for the buildings in the Hayward dataset, which are displayed on the maps included in our presentation.**
<br/><br/>
<font size="4"> 
The final outputs of the code below are stored as Geojson files entitled "hayward_with_predictions.geojson" and "hayward_with_probabilities.geojson" located in the Github repository.
<br/><br/>
Due to the large number of images (over 50,000), running the code below takes many hours, and is thus not recommended to be re-run. If desired, the images can be found at https://drive.google.com/drive/folders/19Y3U00xGvSpq4IcsRoyG6LMpzsG0Swve?usp=sharing. Please note that the file paths below will need to be modified in order for the functions to work.

In [0]:
#Generate predictions for Hayward dataset

In [0]:
#Loads in a portion of the Hayward images (split into 10 groups, labelled 0-9) denoted by input "num"
#Generates binary classifications for the selected images, and saves output as a CSV
def generate_hayward_predictions(num):
  #Model must be loaded in as "loaded_model"
    import os
    import pandas as pd
    hayward_generator = datagen.flow_from_directory(
        '/content/drive/My Drive/Google_API_images/hayward_all/hayward_'+str(num)+'_container',
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary',
        shuffle=False)
    num_files=len(os.listdir('/content/drive/My Drive/Google_API_images/hayward_all/hayward_'+str(num)+'_container/hayward_'+str(num)))
    predictions=loaded_model.predict(hayward_generator,steps=num_files//20+1,verbose=1)
    predictions_classes=predictions>0.5
    predictions_classes=predictions_classes.flatten().astype(int)
    predictions_frame=pd.DataFrame({'File Name':hayward_generator.filenames,'Prediction':predictions_classes})
    predictions_frame.to_csv('/content/drive/My Drive/hayward_predictions/hayward_'+str(num)+'_predictions.csv')

In [0]:
#Loads in a portion of the Hayward images (split into 10 groups, labelled 0-9) denoted by input "num"
#Generates raw model output (probabilities from 0-1) for the selected images, and saves output as a CSV
def generate_hayward_probabilities(num):
  #Model must be loaded in as "loaded_model"
    import os
    import pandas as pd
    hayward_generator = datagen.flow_from_directory(
        '/content/drive/My Drive/Google_API_images/hayward_all/hayward_'+str(num)+'_container',
        target_size=(300, 300),
        batch_size=20,
        class_mode='binary',
        shuffle=False)
    num_files=len(os.listdir('/content/drive/My Drive/Google_API_images/hayward_all/hayward_'+str(num)+'_container/hayward_'+str(num)))
    predictions=loaded_model.predict(hayward_generator,steps=num_files//20+1,verbose=1)
    predictions=predictions.flatten().astype(float)
    predictions_frame=pd.DataFrame({'File Name':hayward_generator.filenames,'Probability':predictions})
    predictions_frame.to_csv('/content/drive/My Drive/hayward_probabilities/hayward_'+str(num)+'_probabilities.csv')

<font size="4"> 
**Running functions above for all 10 sets of images**

In [0]:
import time
for elem in list(range(0,10)):
    generate_hayward_probabilities(elem)
    time.sleep(15)

Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2521s 10s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2532s 10s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2601s 10s/step
Found 0 images belonging to 1 classes.
264/264 [==============================] - 1s 3ms/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2586s 10s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2584s 10s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2602s 10s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2786s 11s/step
Found 0 images belonging to 1 classes.
264/264 [==============================] - 1s 3ms/step
Found 5267 images belonging to 1 classes.
264/264 [==============================] - 2635s 10s/step


In [4]:
#Skipped portions above were re-ran

In [0]:
import time
for elem in list(range(0,10)):
    generate_hayward_predictions(elem)
    time.sleep(15)

Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2501s 9s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2497s 9s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2507s 9s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2495s 9s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2503s 9s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2516s 10s/step
Found 5268 images belonging to 1 classes.
264/264 [==============================] - 2481s 9s/step
Found 5267 images belonging to 1 classes.
264/264 [==============================] - 2480s 9s/step
